In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connecting to sqlite 
conn = sqlite3.connect('fire.db')

# Create a cursor object
cursor = conn.cursor()

In [3]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv('./Data/base_data_1986-2018.csv')

# Write the DataFrame to the SQLite database
df.to_sql('ground_data', conn, if_exists='replace', index=False)

# Commit the changes
conn.commit()

In [4]:
cursor.execute('''SELECT * from ground_data''')
cursor.fetchall()

[('1986-01-16 12:00:00',
  '1986-01',
  -39.05,
  146.34,
  'POINT (146.34 -39.05)',
  584.91650390625,
  5.71484375,
  288.8251953125,
  72.26092529296875,
  2.046238660812378,
  -0.4,
  0.82,
  0.0,
  0.0,
  5831.0,
  5831.0),
 ('1986-01-16 12:00:00',
  '1986-01',
  -38.94,
  146.34,
  'POINT (146.34 -38.94)',
  639.8798828125,
  5.048828125,
  289.13623046875,
  71.379150390625,
  2.046238660812378,
  -0.4,
  0.82,
  0.0,
  1.0,
  5832.0,
  5832.0),
 ('1986-01-16 12:00:00',
  '1986-01',
  -38.94,
  146.45,
  'POINT (146.45 -38.94)',
  640.3271484375,
  5.49609375,
  289.040771484375,
  72.02374267578125,
  2.046238660812378,
  -0.4,
  0.82,
  0.0,
  1.0,
  5942.0,
  5942.0),
 ('1986-01-16 12:00:00',
  '1986-01',
  -38.83,
  145.9,
  'POINT (145.9 -38.83)',
  890.787109375,
  5.88671875,
  289.68359375,
  69.36932373046875,
  -0.2895850241184234,
  -0.4,
  0.82,
  0.0,
  0.0,
  5393.0,
  5393.0),
 ('1986-01-16 12:00:00',
  '1986-01',
  -38.83,
  146.01,
  'POINT (146.01 -38.83)',
  8

In [5]:
#Get all tables
def get_all_tables(cursor):
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor.execute(sql_query)
    return cursor.fetchall()

# Get column names and data types for each table
def get_table_info(cursor, table_name):
    cursor.execute(f'PRAGMA table_info({table_name})')
    return cursor.fetchall()

In [6]:
tables = [table[0] for table in get_all_tables(cursor)]

for table in tables:
    print(f"Table: {table}")
    for column in get_table_info(cursor, table):
        print(f"Column: {column[1]}, Type: {column[2]}")
    print("\n")

Table: ground_data
Column: time, Type: TEXT
Column: month, Type: TEXT
Column: lat, Type: REAL
Column: lon, Type: REAL
Column: geometry, Type: TEXT
Column: mrso, Type: REAL
Column: sfcWind, Type: REAL
Column: tas, Type: REAL
Column: hurs, Type: REAL
Column: pdsi, Type: REAL
Column: MEI, Type: REAL
Column: SOI, Type: REAL
Column: bushfire_count, Type: REAL
Column: prescribed_count, Type: REAL
Column: grid_id, Type: REAL
Column: index_right, Type: REAL


